In [ ]:
"""
Purpose: To put all of the rules together
to autoatically proofread a cell


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

INFO - 2021-02-03 07:54:52,375 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-03 07:54:52,376 - settings - Setting database.user to celiib
INFO - 2021-02-03 07:54:52,377 - settings - Setting database.password to newceliipass
INFO - 2021-02-03 07:54:52,388 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-03 07:54:52,389 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-03 07:54:52,401 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-03 07:54:52,673 - settings - Setting enable_python_native_blobs to True


In [8]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-03 07:55:03,435 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-03 07:55:03,733 - settings - Setting enable_python_native_blobs to True


# Importing the Neuron

In [9]:
import datajoint as dj
dj.config["display.limit"] = 40
curr_table = minnie.Decomposition & "process_version = 4" & "n_limbs>5" & "n_error_limbs>0" & "n_somas>1"
curr_table

INFO - 2021-02-03 07:55:03,963 - settings - Setting display.limit to 40


segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135012398582,0,0.25,30.00,4,0,1,=BLOB=,1376990,2794781,0,3,1,3,2,7,332,193,13857599.090995071,7698499.767603829,24782.936,182.056,175.907,512.504,498.037,5073,0.00036608073062934334,15.28012048192771,9021770.899,184,0.0005623064536655776,27.571,1130755898564.6187,149139625.2549947,81598.25458505328,125336.35704382107,6145412492.199015,18888.6673
864691135065018692,0,0.25,30.00,4,0,1,=BLOB=,927599,1896021,0,2,2,1,2,19,275,62,10134692.441118384,2065744.7931319945,17371.879,110.248,110.248,544.807,544.758,1859,0.0001834293453699376,6.76,5876271.812,132,0.00031635704737206256,14.083,333744826425.14685,116071713.89832559,32930.92793532444,56795.33505301828,2528369897.1602035,13467.9901
864691135065032772,0,0.25,30.00,4,0,1,=BLOB=,1284918,2610256,0,1,0,1,3,19,339,49,12070580.98087547,2067456.6676802342,24476.944,293.07,258.608,496.883,455.767,4680,0.00038771953126489554,13.805309734513274,9141427.307,231,0.0005119550637805012,20.26,984274848135.3904,134477821.95931625,81543.28691343585,107671.90013989249,4260930078.5081835,10872.3472
864691135210078912,0,0.25,30.00,4,0,1,=BLOB=,2254579,4581171,0,1,1,1,3,19,680,410,25709086.238637865,14910186.433055326,22806.349,91.868,91.868,488.977,396.491,7956,0.00030946257389899087,11.7,14370940.599,305,0.0005536172072518077,26.085,1530423053724.611,123769896.05576578,59528.48885871943,106494.2856858726,5017780504.015119,21160.2523
864691135345274271,0,0.25,30.00,4,0,1,=BLOB=,2309163,4637079,0,3,3,1,3,23,568,232,20083889.41475576,7902329.442060159,19837.799,307.913,275.329,552.594,537.217,6717,0.00033444717112737023,11.825704225352112,16093790.786,398,0.00041736593257090944,16.877,1327097700929.9097,126146626.59829712,66077.7239668967,82460.2306924701,3334416333.994748,10629.5786
864691135373436616,0,0.25,30.00,4,0,1,=BLOB=,1785657,3619863,0,5,5,1,2,20,493,131,18421878.018574927,4845043.772362778,26060.468,261.01

In [10]:
segment_id = 864691135654121154 #was very big and hard to visualize
segment_id = 864691135065018692 #worked
segment_id = 864691135526113627 #worked
segment_id = 864691135403709422 #worked
segment_id = 864691136390485119

In [11]:
neuron_obj_original = (curr_table & dict(segment_id=segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [ ]:
#nviz.visualize_neuron_lite(neuron_obj_original)

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj_original,
                     limb_branch_dict=dict(L1="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj_original)

# Developing the Overall Proofreading Function

In [43]:
pru = reload(pru)

In [44]:
output_list = pru.proofread_neuron(
    
    input_neuron = neuron_obj_original,

    attempt_to_split_neuron = True,
    plot_neuron_split_results = False,

    plot_neuron_before_filtering = False,

    plot_axon = False,
    plot_axon_like = False,

    # -- for the filtering loop
    plot_limb_branch_filter_with_disconnect_effect = True,
    plot_final_filtered_neuron = True,

    # -- for the output --
    return_process_info = True,

    debug_time = True,
    verbose = False,
    verbose_outline=True

    )

---- Part A: Attempting to split neuron --------



Time for Split Neuron = 159.1078279018402
 # of neurons found after split= 2

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 23.536929845809937


 ------ Part C: Inhibitory Excitatory Classification ---- 


Did not pass final filters to continuing
Inhibitory Excitatory Classification = 12.031546115875244
cell type = excitatory


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 1:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 2:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': True, 'skip_double_back_errors_for_axon': True, 'double_back_threshold': 140}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 4:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True}



--- Working on filter 5:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 30}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh





 ---- Total time for applying filter: 100.49672389030457 -----

 Working on visualization type: mesh



--- Working on Neuron 1 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 27.729567527770996


 ------ Part C: Inhibitory Excitatory Classification ---- 


Inhibitory Excitatory Classification = 18.28194570541382
cell type = inhibitory


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 1:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True}



--- Working on filter 2:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 30}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': False, 'skip_double_back_errors_for_axon': True}





 ---- Total time for applying filter: 74.92204093933105 -----

 Working on visualization type: mesh


In [48]:
output_list[0]["filtered_neuron"].description

'0_25_soma_0_split_soma_0_split_soma_0_split_soma_0_split'

# Figuring Out the Closest Nuclei

In [ ]:
"""




"""